In [1]:
import os

from pandas import to_datetime
from dateutil.relativedelta import relativedelta

currentDirectory = os.getcwd()
projectDirectory = os.path.dirname(currentDirectory)

# Modify this to change the output file name.
reportName = 'Force Stat Report'
presentationName = 'Overview'
templateName = 'Template'

reportPath = f'{projectDirectory}\\output\\{reportName}.xlsx'
presentationPath = f'{projectDirectory}\\output\\{presentationName}.pptx'
templatePath = f'{projectDirectory}\\template\\{templateName}.pptx'

# Modify this to get a CityWide or list of boroughs.
cw = True
boroList = ['PBBN']

# Modify to change the date range.
# Dates are in the format YYYY-MM-DD
dateStartCurr = to_datetime("2024-06-10")
dateEndCurr = to_datetime("2024-07-07")
ytdCurr = to_datetime("2024-01-01")

dateStartPre = dateStartCurr - relativedelta(years=1)
dateEndPre = dateEndCurr - relativedelta(years=1)
ytdPre = ytdCurr - relativedelta(years=1)

# Modify to change the path of the data files.
incallPath = f'{projectDirectory}\\Data\\csv\\INCALL.csv'
intallPath = f'{projectDirectory}\\Data\\csv\\INTALL.csv'
arrestsPath = f'{projectDirectory}\\Data\\csv\\ARRESTS.csv'

try:
    if not cw and not boroList:
        raise Exception("Please select CityWide or a borough.")

    if os.path.exists(f"{reportPath}"):
        os.remove(f"{reportPath}")
        print("Removed old output file.\n")
    else:
        print("File does not exist. Creating new file.\n")
except PermissionError:
    raise Exception("Output file must be closed to run the program.")


Removed old output file.



In [2]:
import read

print('Reading files...')
incall, intall, arrests = read.readData(incallPath, intallPath, arrestsPath)
print('Done reading.\n')


Reading files...
Done reading.



In [3]:
import clean

print('Cleaning incidents...')
incall = clean.cleanINCALL(incall)
print('Cleaned incidents.\n')

print('Cleaning interactions...')
intall = clean.cleanINTALL(intall)
print('Cleaned interactions.\n')

print('Cleaning arrests...')
arrests = clean.cleanARRESTS(arrests)
print('Cleaned arrests.\n')

Cleaning incidents...
Cleaned incidents.

Cleaning interactions...
Cleaned interactions.

Cleaning arrests...
Cleaned arrests.



In [4]:
import reset

print('Setting date filters...')

(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD
) = reset.reset(incall,intall,arrests,dateStartCurr,dateEndCurr,dateStartPre,dateEndPre,ytdCurr,ytdPre)

print('Date filters set.\n')

Setting date filters...
Date filters set.



In [5]:
from pandas import DataFrame

dateDict = { 'Date Start:' : [dateStartCurr.strftime('%m/%d/%Y')], 'Date End:' : [dateEndCurr.strftime('%m/%d/%Y')]}
dateDataFrame = DataFrame(dateDict)
dateDataFrame.index = ['Date:']
dateDataFrame.index.name = 'Period'

In [6]:
import cwSummary
import cwBreakdown

cwTablesDict = {}

if cw == True:
    print('Building CityWide tables...')

    summaryDataFrames = cwSummary.periodSummary(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
    previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
    currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD)

    breakdownDataFrames = cwBreakdown.periodBreakdown(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentArrestPeriod)

    summaryDataFrames = [dateDataFrame] + summaryDataFrames
    breakdownDataFrames = [dateDataFrame] + breakdownDataFrames

    cwTablesDict["CityWide Period Summary"] = summaryDataFrames
    cwTablesDict["CityWide Period Breakdown"] = breakdownDataFrames

    print("CityWide tables built.\n")

Building CityWide tables...
CityWide tables built.



In [7]:
import overviewPpt

overviewPpt.fillPresentation(templatePath, presentationPath, summaryDataFrames, breakdownDataFrames, dateStartCurr, dateEndCurr)

In [8]:
import importlib
import boroSummary
import boroBreakdown
importlib.reload(boroBreakdown)

boroTablesDict = {}

for boro in boroList:
    print('Building borough tables...')

    summaryBoroDataFrames = boroSummary.boroSummary(boro, currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
                    previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
                    currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD)
    
    breakdownBoroDataFrames = boroBreakdown.boroBreakdown(boro, currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentArrestPeriod)

    summaryBoroDataFrames = [dateDataFrame] + summaryBoroDataFrames
    breakdownBoroDataFrames = [dateDataFrame] + breakdownBoroDataFrames

    boroTablesDict[f"{boro} Summmary"] = summaryBoroDataFrames
    boroTablesDict[f"{boro} Breakdown"] = breakdownBoroDataFrames

    print("Borough tables built.\n")

Building borough tables...
Borough tables built.



In [9]:
import output

outputDict = cwTablesDict | boroTablesDict

print("Outputting to Excel...")

output.outputToExcel(outputDict, reportPath)

print(f"Output to {reportPath}\n")

Outputting to Excel...
Output to c:\Users\robwo\Desktop\CS\nypd\Python\ForceStatGen\output\Force Stat Report.xlsx

